In [1]:
import numpy as np
import pyopencl as cl
import pyopencl.array
from pyopencl.elementwise import ElementwiseKernel

In [2]:
a_np = np.random.randn(50000).astype(np.float32)
b_np = np.random.randn(50000).astype(np.float32)

In [3]:
ctx = cl.create_some_context()
queue = cl.CommandQueue(ctx)

In [4]:
a_g = cl.array.to_device(queue, a_np)
b_g = cl.array.to_device(queue, b_np)

In [5]:
lin_comb = ElementwiseKernel(context = ctx,
    arguments = "float k1, float *a_g, float k2, float *b_g, float *res_g",
    operation = "res_g[i] = k1 * a_g[i] + k2 * b_g[i]",
    name = "lin_comb")

In [6]:
res_g = cl.array.empty_like(a_g)
lin_comb(2, a_g, 3, b_g, res_g)

In [7]:
# Check on GPU with PyOpenCL Array:
print((res_g - (2 * a_g + 3 * b_g)).get())

[0. 0. 0. ... 0. 0. 0.]


In [8]:
# Check on CPU with Numpy:
res_np = res_g.get()
print(res_np - (2 * a_np + 3 * b_np))
print(np.linalg.norm(res_np - (2 * a_np + 3 * b_np)))

[0. 0. 0. ... 0. 0. 0.]
0.0
